In [1]:
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import librosa
from tqdm import tqdm
from sklearn.metrics import roc_curve


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

SAMPLE_RATE = 16000
SEGMENT_LEN = 32000
FRAME_SIZE = 160

BATCH_SIZE = 1
LR = 1e-4
EPOCHS = 10

os.makedirs("checkpoints", exist_ok=True)
DEVICE

Using device: cuda


'cuda'

In [3]:
def circular_shift(arr, shift):
    return np.roll(arr, shift)

def least_number(a, b):
    g = math.gcd(a, b)
    return a // g, b // g
def generate_matrix(N, M, P, Q):
    row = np.concatenate([np.ones(M), np.zeros(N - M)])
    rows = []
    for _ in range(M):
        rows.append(row)
        row = circular_shift(row, M)
    return np.array(rows)
def apply_cs(audio, sensing_matrix, frame_size):
    n_frames = len(audio) // frame_size
    out = []
    for i in range(n_frames):
        frame = audio[i*frame_size:(i+1)*frame_size]
        out.append(sensing_matrix @ frame)
    return np.concatenate(out)


In [4]:
class ASVspoofCSDataset(Dataset):
    def __init__(self, audio_dir, protocol_path, apply_cs=True):
        self.audio_dir = audio_dir
        self.apply_cs = apply_cs

        # ===== 25% Compressive Sensing =====
        N = FRAME_SIZE                  # original frame length (160)
        M = int(0.25 * N)               # 25% measurements (40)

        P, Q = least_number(M, N)
        self.sensing_matrix = generate_matrix(N, M, P, Q)
        # ==================================

        available_files = set(
            os.path.splitext(f)[0]
            for f in os.listdir(audio_dir)
            if f.endswith(".flac")
        )

        self.items = []
        skipped = 0

        with open(protocol_path) as f:
            for line in f:
                parts = line.strip().split()
                utt = parts[1]                         # CRITICAL
                label = 1 if parts[-1] == "spoof" else 0

                if utt not in available_files:
                    skipped += 1
                    continue

                self.items.append((utt, label))

        print(f"[Dataset] Loaded {len(self.items)} | Skipped {skipped}")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        utt, label = self.items[idx]
        audio_path = os.path.join(self.audio_dir, utt + ".flac")
        
        audio, _ = librosa.load(audio_path, sr=SAMPLE_RATE)

        if len(audio) < SEGMENT_LEN:
            audio = np.pad(audio, (0, SEGMENT_LEN - len(audio)))
        else:
            audio = audio[:SEGMENT_LEN]

        if self.apply_cs:
            audio = apply_cs(audio, self.sensing_matrix, FRAME_SIZE)

        return torch.tensor(audio, dtype=torch.float32).unsqueeze(0), torch.tensor(label)



In [8]:
#RAWNET-2
class FixedSincConv(nn.Module):
    def __init__(self, out_channels=128, kernel_size=129, sample_rate=16000):
        super().__init__()

        if kernel_size % 2 == 0:
            kernel_size += 1

        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.sample_rate = sample_rate

        # Linear-scale fixed filterbank (S3 setup in paper)
        low_hz = 0
        high_hz = sample_rate // 2
        hz = torch.linspace(low_hz, high_hz, out_channels + 1)

        self.low = hz[:-1]
        self.high = hz[1:]

        n = torch.arange(-(kernel_size // 2), kernel_size // 2 + 1)
        self.register_buffer("n", n)
        self.register_buffer("window", torch.hamming_window(kernel_size))

    def forward(self, x):
        device = x.device
        filters = []

        for low, high in zip(self.low, self.high):
            f1 = low / self.sample_rate
            f2 = high / self.sample_rate
            n = self.n.to(device)

            sinc1 = torch.sin(2 * math.pi * f1 * n) / (n + 1e-9)
            sinc2 = torch.sin(2 * math.pi * f2 * n) / (n + 1e-9)
            band = (sinc2 - sinc1) * self.window.to(device)
            filters.append(band)

        filters = torch.stack(filters).view(self.out_channels, 1, self.kernel_size)
        return F.conv1d(x, filters, stride=1, padding=self.kernel_size // 2)

class FMS(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.fc = nn.Linear(channels, channels)

    def forward(self, x):
        # x: (B, C, T)
        s = x.mean(dim=2)               # Global average pooling
        s = torch.sigmoid(self.fc(s))   # (B, C)
        s = s.unsqueeze(2)
        return x * s + s
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()

        self.conv1 = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(channels)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(channels)

        self.fms = FMS(channels)
        self.pool = nn.MaxPool1d(3)

    def forward(self, x):
        residual = x
        x = F.leaky_relu(self.bn1(self.conv1(x)), 0.3)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.3)
        x = x + residual
        x = self.pool(x)
        x = self.fms(x)
        return x
class RawNet2_AntiSpoof(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()

        # Front-end
        self.sinc = FixedSincConv(out_channels=128, kernel_size=129)
        self.bn0 = nn.BatchNorm1d(128)

        # Residual blocks (128 × 2)
        self.block1 = nn.Sequential(
            ResidualBlock(128),
            ResidualBlock(128)
        )

        # Channel expansion
        self.conv_expand = nn.Conv1d(128, 512, kernel_size=1)

        # Residual blocks (512 × 4)
        self.block2 = nn.Sequential(
            ResidualBlock(512),
            ResidualBlock(512),
            ResidualBlock(512),
            ResidualBlock(512)
        )

        # GRU
        self.gru = nn.GRU(
            input_size=512,
            hidden_size=1024,
            batch_first=True
        )

        # Classifier
        self.fc1 = nn.Linear(1024, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # x: (B, 1, 64000)
        x = self.sinc(x)
        x = F.leaky_relu(self.bn0(x), 0.3)

        x = self.block1(x)
        x = self.conv_expand(x)
        x = self.block2(x)

        # (B, C, T) → (B, T, C)
        x = x.permute(0, 2, 1)

        self.gru.flatten_parameters()
        x, _ = self.gru(x)

        x = x[:, -1, :]  # last frame
        x = F.leaky_relu(self.fc1(x), 0.3)
        return self.fc2(x)


In [9]:
def compute_eer(scores, labels):
    fpr, tpr, _ = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    return 100 * fpr[np.nanargmin(np.abs(fnr - fpr))]
def compute_min_tDCF(scores, labels):
    Ptar, Cmiss, Cfa = 0.01, 1, 1
    fpr, tpr, _ = roc_curve(labels, scores, pos_label=1)
    fnr = 1 - tpr
    return np.min(Cmiss * Ptar * fnr + Cfa * (1 - Ptar) * fpr)


In [10]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for x, y in tqdm(loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    scores, labels = [], []
    for x, y in loader:
        probs = torch.softmax(model(x.to(DEVICE)), dim=1)
        scores.extend(probs[:, 1].cpu().numpy())
        labels.extend(y.numpy())
    return np.array(scores), np.array(labels)


In [11]:
train_set = ASVspoofCSDataset(
    "/kaggle/input/la-datasets/LA/ASVspoof2019_LA_train/flac",
    "/kaggle/input/la-datasets/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt",
    apply_cs=True
)

dev_set = ASVspoofCSDataset(
    "/kaggle/input/la-datasets/LA/ASVspoof2019_LA_dev/flac",
    "/kaggle/input/la-datasets/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt",
    apply_cs=True
)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
dev_loader = DataLoader(dev_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model = RawNet2_AntiSpoof().to(DEVICE)
criterion = nn.CrossEntropyLoss()

# model = AASIST().to(DEVICE)
# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

best_eer = 100


[Dataset] Loaded 25380 | Skipped 0
[Dataset] Loaded 24844 | Skipped 0


In [12]:
PATIENCE = 7  # typical: 5–15
best_eer = float("inf")
patience_counter = 0

for epoch in range(EPOCHS):
    loss = train_epoch(model, train_loader, optimizer, criterion)
    scores, labels = evaluate(model, dev_loader)

    eer = compute_eer(scores, labels)
    tdcf = compute_min_tDCF(scores, labels)

    print(
        f"Epoch {epoch+1} | "
        f"Loss {loss:.4f} | "
        f"EER {eer:.2f}% | "
        f"min-tDCF {tdcf:.4f}"
    )

    # 🔥 Improvement → save + reset patience
    if eer < best_eer:
        best_eer = eer
        patience_counter = 0

        torch.save(
            {
                "epoch": epoch + 1,
                "model": model.state_dict(),
                "eer": eer,
                "tdcf": tdcf
            },
            "/kaggle/working/checkpoints/rawnet2_cs_best.pth"
        )
        print(f"✅ New best EER: {eer:.2f}% — model saved")

    # 🛑 No improvement → increase patience
    else:
        patience_counter += 1
        print(
            f"⏳ No improvement "
            f"({patience_counter}/{PATIENCE})"
        )

        if patience_counter >= PATIENCE:
            print(
                f"🛑 Early stopping triggered at epoch {epoch+1}. "
                f"Best EER: {best_eer:.2f}%"
            )
            break


100%|██████████| 25380/25380 [15:30<00:00, 27.29it/s]


Epoch 1 | Loss 0.2931 | EER 37.99% | min-tDCF 0.0093
✅ New best EER: 37.99% — model saved


100%|██████████| 25380/25380 [15:07<00:00, 27.98it/s]


Epoch 2 | Loss 0.2535 | EER 31.08% | min-tDCF 0.0081
✅ New best EER: 31.08% — model saved


100%|██████████| 25380/25380 [15:05<00:00, 28.03it/s]


...

Epoch 8 | Loss 0.0918 | EER 23.23% | min-tDCF 0.0075
✅ New best EER: 23.23% — model saved


100%|██████████| 25380/25380 [15:10<00:00, 27.88it/s]


Epoch 9 | Loss 0.0788 | EER 22.49% | min-tDCF 0.0075
✅ New best EER: 22.49% — model saved


100%|██████████| 25380/25380 [15:08<00:00, 27.93it/s]


Epoch 10 | Loss 0.0708 | EER 28.61% | min-tDCF 0.0087
⏳ No improvement (1/7)


In [13]:
import zipfile
import os

zip_path = "/kaggle/working/checkpointsd.zip"
folder_path = "/kaggle/working/checkpoints"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            arcname = os.path.relpath(full_path, folder_path)
            zipf.write(full_path, arcname)

print("Zipped to:", zip_path)

Zipped to: /kaggle/working/checkpointsd.zip
